# Cube Operations

This test case is based on a user story from Chris Kent in Applied Science.  He stated that a plot combination he frequently requires is one global plot, and a plot showing an associated curve following the dimensional sum or average of the cube.

In [ ]:
import cartopy.crs as ccrs
import iris
#import iris.coord_categorisation as ccat
import iris.plot as iplt
import iris.quickplot as qplt

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
% matplotlib inline

In [ ]:
# This is probably going to be most useful shown with a 4-dimensional cube.  
# That way it can still be interactiverized with one slider while the sum or average is static in another.
# Here it will all be slicing though.

In [ ]:
cube = iris.load_cube(iris.sample_data_path('uk_hires.pp'), 'air_potential_temperature')
print cube

In [ ]:
# In the cube selected here, there would be two sliders; one for time and one for model_level_number.
# The time vs phenomenon plot would be invariant with time, but the presence of the slider should not break the plot.

In [ ]:
mean_cube = cube.collapsed('time', iris.analysis.MEAN)
print mean_cube

In [ ]:
fig = plt.figure(figsize = [12, 9])

ax1 = plt.subplot(2,2,1, projection=ccrs.OSGB())
ax1.coastlines()
ax1.set_global()
ax1.set_title('OSGB View of Data')
iplt.pcolormesh(cube[0, 0])

ax2 = plt.subplot(2,2,2, projection=ccrs.Mercator())
ax2.set_xticks(cube.coord('grid_longitude').points[0:187:10]) # Why doesn't this work?
ax2.set_title('Air Potential Temperature')
iplt.pcolormesh(cube[0, 0])

ax3 = plt.subplot(2,2,3, colspan=2)
ax3.set_title('Mean over Time of Air Potential Temperature')
iplt.plot(mean_cube[0, 0])

fig.tight_layout()
iplt.show()